In [1]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

tf.set_random_seed(777)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import requests
url = 'https://biendata.com/competition/airquality/bj/2018-03-01-0/2018-05-31-23/2k0d1d8'
respones= requests.get(url)
with open ("bj_airquality.csv",'w') as f:
    f.write(respones.text)

In [3]:
import pandas as pd
b_air = pd.read_csv("bj_airquality.csv")

In [6]:
b_air = b_air[b_air['time'] < '2018-05-14 00:00:00']

In [4]:
b_air.head(3)

,id,station_id,time,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration
0,2941450,dongsi_aq,2018-03-31 07:00:00,105.0,172.0,53.0,0.8,127.0,14.0
1,2941451,tiantan_aq,2018-03-31 07:00:00,95.0,123.0,54.0,0.9,121.0,15.0
2,2941452,guanyuan_aq,2018-03-31 07:00:00,95.0,139.0,66.0,0.8,123.0,13.0


In [7]:
b_air.tail(3)

,id,station_id,time,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration
34647,2998338,xizhimenbei_aq,2018-05-13 23:00:00,126.0,137.0,72.0,1.0,34.0,5.0
34648,2998339,nansanhuan_aq,2018-05-13 23:00:00,145.0,194.0,98.0,1.5,9.0,8.0
34649,2998340,dongsihuan_aq,2018-05-13 23:00:00,172.0,NaN,102.0,1.6,5.0,14.0


In [8]:
b_air.columns = ['id', 'test_id', 'time', 'PM2.5', 'PM10','NO2', 'CO', 'O3', 'SO2']

In [9]:
sample = pd.read_csv("sample_submission.csv")

def id(i):
    return i.split('#')[0]

test_station = pd.DataFrame(list(map(id, sample['test_id']))).drop_duplicates()
test_station.columns = ['id']
test_station = list(test_station['id'])

bj_test_station = test_station[:35]
ld_test_station = test_station[35:]

In [10]:
def function1(i, y):
    b_air[b_air["test_id"] == i][y].fillna(np.mean(b_air[b_air["test_id"] == i][y]))
    dataset = b_air[b_air["test_id"] == i][y]
    return dataset

In [11]:
def lstm_cell():
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_cell_hidden_dim, 
                                        forget_bias=forget_bias, state_is_tuple=True, activation=tf.nn.softsign)
    if keep_prob < 1.0:
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
    return cell

In [12]:
def smape(targets, predictions):
    dividend= np.abs(np.array(targets) - np.array(predictions))
    denominator = np.array(targets) + np.array(predictions)
    
    return 2 * np.mean(np.divide(dividend, denominator, out=np.zeros_like(dividend), where=denominator!=0, casting='unsafe'))

In [13]:
def min_max_scaling(x):
    x_np = np.asarray(x)
    return (x_np-x_np.min()) / (x_np.max()-x_np.min() + 1e-7)

In [33]:
# 하이퍼파라미터
rnn_cell_hidden_dim = 32   # 각 셀의 (hidden)출력 크기
forget_bias = 1.0          # 망각편향(기본값 1.0)
num_stacked_layers = 1     # stacked LSTM layers 개수
keep_prob = 1.0            # dropout할 때 keep할 비율
epoch_num = 1000           # 에폭 횟수(학습용전체데이터를 몇 회 반복해서 학습할 것인가 입력)
learning_rate = 0.3        # 학습률

In [34]:
tf.reset_default_graph()

In [ ]:
submission = []
y = 'O3'
for idx,i in enumerate(bj_test_station):
    mean = np.mean(pd.DataFrame(function1(i,y)))
    aq1 = list(pd.DataFrame(function1(i,y)).fillna(round(mean,1))[y])
    #aq = min_max_scaling(aq1)
    
    data=[]
    for i in range(len(aq1)-48):
        aq = aq1[i:i+48]
        data.append(aq)
    
    data_X=[]
    data_Y=[]
    seq_length = 7
    for i in range(len(data)-seq_length):
        data_X.append(data[i:i+seq_length]) 
        data_Y.append(data[i+seq_length]) # i+seq_length번째 날 예측
        
        
    train_size = int(len(data_Y) * 0.8)
    # 데이터를 잘라 학습용 데이터 생성
    trainX = data_X[:train_size]
    trainY = data_Y[:train_size]

    # 데이터를 잘라 테스트용 데이터 생성
    testX = data_X[train_size:]
    testY = data_Y[train_size:]
    
    trainX = np.reshape(trainX, [len(trainX),seq_length,48])
    testX = np.reshape(testX, [len(testX),seq_length,48])
    trainY = np.reshape(trainY,[len(trainY),48])
    testY = np.reshape(testY,[len(testY),48])

    # 모델
    X = tf.placeholder(tf.float32, [None, seq_length, 48])
    Y = tf.placeholder(tf.float32, [None, 48])

    targets = tf.placeholder(tf.float32, [None, 48])
    predictions = tf.placeholder(tf.float32, [None, 48])

    # num_stacked_layers개의 층으로 쌓인 Stacked RNNs 생성
    stackedRNNs = [lstm_cell() for _ in range(num_stacked_layers)]
    multi_cells = tf.contrib.rnn.MultiRNNCell(stackedRNNs, state_is_tuple=True) if num_stacked_layers > 1 else lstm_cell()

    # RNN Cell(여기서는 LSTM셀임)들을 연결
    hypothesis, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)
    print("hypothesis: ", hypothesis)

    hypothesis = tf.contrib.layers.fully_connected(hypothesis[:, -1], 48, activation_fn=tf.nn.relu)
    
    # 손실함수로 평균제곱오차를 사용한다
    loss = tf.reduce_sum(tf.square(hypothesis - Y))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)
    
    #rmse = tf.sqrt(tf.reduce_mean(tf.squared_difference(targets, predictions)))
    temp = tf.divide(tf.abs(targets-predictions), targets+predictions)
    smape = 2*tf.reduce_mean(temp)
    
    train_error_summary = []
    test_error_summary = []  # 테스트용 데이터의 오류를 중간 중간 기록한다
    test_predict = ''        # 테스트용데이터로 예측한 결과
 
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # 학습한다
    print('학습을 시작합니다...')

    for epoch in range(epoch_num):
        _, _loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
        if ((epoch+1) % 100 == 0) or (epoch == epoch_num-1): # 100번째마다 또는 마지막 epoch인 경우

            # 학습용데이터로 rmse오차를 구한다
            train_predict = sess.run(hypothesis, feed_dict={X: trainX})
            train_error = sess.run(smape, feed_dict={targets: trainY, predictions: train_predict})
            train_error_summary.append(train_error)
 
            # 테스트용데이터로 rmse오차를 구한다
            test_predict = sess.run(hypothesis, feed_dict={X: testX})
            test_error = sess.run(smape, feed_dict={targets: testY, predictions: test_predict})
            test_error_summary.append(test_error)
        
            print("epoch: {}, train_error: {}, test_error: {}".format(epoch+1,train_error,test_error))
    
    recent_data = np.reshape(data_X[len(data_X)-seq_length:][0], [-1,seq_length,48])
    test_predict = sess.run(hypothesis, feed_dict={X: recent_data})
    #print("test_predict", test_predict)
     
    submission.append(test_predict[0])
    
    tf.reset_default_graph()

hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)
학습을 시작합니다...
epoch: 100, train_error: 0.533118486404419, test_error: 0.6355494260787964
epoch: 200, train_error: 0.5057744383811951, test_error: 0.6039650440216064
epoch: 300, train_error: 0.5039263963699341, test_error: 0.5959115624427795
epoch: 400, train_error: 0.5063007473945618, test_error: 0.5967052578926086
epoch: 500, train_error: 0.505075216293335, test_error: 0.5920022130012512
epoch: 600, train_error: 0.5055768489837646, test_error: 0.5931032299995422
epoch: 700, train_error: 0.506637454032898, test_error: 0.5920045375823975
epoch: 800, train_error: 0.47614607214927673, test_error: 0.5568923950195312
epoch: 900, train_error: 0.46139422059059143, test_error: 0.5354474782943726
epoch: 1000, train_error: 0.4473627209663391, test_error: 0.5140455961227417
hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)
학습을 시작합니다...
epoch: 100, train_error: 0.6254305243492126, test_error: 0.7090994

학습을 시작합니다...
epoch: 100, train_error: 0.4792934060096741, test_error: 0.5621533393859863
epoch: 200, train_error: 0.4777783155441284, test_error: 0.5612008571624756
epoch: 300, train_error: 0.47904708981513977, test_error: 0.5617205500602722
epoch: 400, train_error: 0.4790411591529846, test_error: 0.561745285987854
epoch: 500, train_error: 0.47855570912361145, test_error: 0.5604842901229858
epoch: 600, train_error: 0.4790785312652588, test_error: 0.5617890357971191
epoch: 700, train_error: 0.4790806174278259, test_error: 0.561794102191925
epoch: 800, train_error: 0.479235976934433, test_error: 0.5621341466903687
epoch: 900, train_error: 0.4790799021720886, test_error: 0.5617927312850952
epoch: 1000, train_error: 0.47910138964653015, test_error: 0.5618190169334412
hypothesis:  Tensor("rnn/transpose_1:0", shape=(?, 7, 32), dtype=float32)
학습을 시작합니다...
epoch: 100, train_error: 0.6336760520935059, test_error: 0.7366752028465271
epoch: 200, train_error: 0.6360079646110535, test_error: 0.7389

In [22]:
submission[8]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

In [23]:
# nan....
#submission[8]
submission[8] = np.array(np.mean(pd.DataFrame(submission).fillna(0)))

In [19]:
PM25 = np.reshape(submission,-1)
PM25

array([157.82441711, 159.66041565, 160.50166321, ...,  51.91577148,
        49.79025269,  48.53936768])

In [24]:
PM10 = np.reshape(submission,-1)
PM10

array([128.67303467, 128.09837341, 127.09113312, ..., 149.79852295,
       149.59429932, 149.33416748])

In [29]:
O3 = np.reshape(submission,-1)
O3

array([91.8210907 , 89.62960052, 85.80215454, ..., 63.50445557,
       53.46598816, 44.10897064])

In [30]:
pm25 = pd.DataFrame(PM25)
pm10 = pd.DataFrame(PM10)
o3 = pd.DataFrame(O3)

In [31]:
pm25.to_csv('pm25_0513.csv')
pm10.to_csv('pm10_0513.csv')
o3.to_csv('o3_0513.csv')

In [32]:
p25 = pd.read_csv('ld_25_0513.csv')
p10 = pd.read_csv('ld_10_0513.csv')

In [33]:
pm25.columns = ['y']
pm10.columns = ['y']
o3.columns = ['y']
p25.columns = ['a', 'y']
p10.columns = ['a', 'y']

In [34]:
len(p25)

624

In [35]:
sample['PM2.5'] = list(pm25['y'])+(list(p25['y']))

In [36]:
sample['PM10'] = list(pm10['y'])+(list(p10['y']))

In [37]:
sample['O3'][:1680] = list(o3['y'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
bj_test_station[8]

'zhiwuyuan_aq'

In [39]:
sample.to_csv('sample_0513.csv',index=False)

In [40]:
import requests
files = {'files': open('sample_0513.csv','rb')}
data = {
    "user_id": "dasolhwang",
    "team_token": "53ed3560830874472182ca33790fe582ebd15bfb2f0506cb03d7757a9acc7480",
    "description": 'dasol_2018-05-13',
    "filename": "sample_0513.csv",
}
url = 'https://biendata.com/competition/kdd_2018_submit/'
response = requests.post(url, files=files, data=data)
print(response.text)

{"success": true}
